In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense 
from tensorflow.keras.layers import GRU, Dense

2024-04-01 15:21:47.694870: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-04-01 15:21:47.694927: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
df = pd.read_csv('srednjeni_podaci.csv', parse_dates=True)

In [3]:
df = df.drop(df.columns[0], axis=1)

In [4]:
time_series = df['prosek'].values.astype(float)

In [5]:
scaler = MinMaxScaler(feature_range=(0, 1))
time_series_normalized = scaler.fit_transform(time_series.reshape(-1, 1))

In [6]:
def create_dataset(time_series, time_steps):
    X, y = [], []
    for i in range(len(time_series) - time_steps):
        X.append(time_series[i:(i + time_steps)])
        y.append(time_series[i + time_steps])
    return np.array(X), np.array(y)

In [7]:
time_steps = 12  
batch_size = 32
epochs = 100


X, y = create_dataset(time_series_normalized, time_steps)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
model = Sequential([
    GRU(50, input_shape=(time_steps, 1)),
    Dense(1)
])

# Kompilacija modela
model.compile(optimizer='adam', loss='mean_squared_error')

# Treniranje modela
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=1)

2024-04-01 15:21:53.253004: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2024-04-01 15:21:53.253052: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2024-04-01 15:21:53.253094: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (marija-VivoBook-ASUSLaptop-X512DA-X512DA): /proc/driver/nvidia/version does not exist
2024-04-01 15:21:53.254094: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/100
3/3 [==============================] - 4s 14ms/step - loss: 0.2753
Epoch 2/100
3/3 [==============================] - 0s 18ms/step - loss: 0.1880
Epoch 3/100
3/3 [==============================] - 0s 15ms/step - loss: 0.1203
Epoch 4/100
3/3 [==============================] - 0s 13ms/step - loss: 0.0700
Epoch 5/100
3/3 [==============================] - 0s 12ms/step - loss: 0.0440
Epoch 6/100
3/3 [==============================] - 0s 19ms/step - loss: 0.0372
Epoch 7/100
3/3 [==============================] - 0s 12ms/step - loss: 0.0408
Epoch 8/100
3/3 [==============================] - 0s 14ms/step - loss: 0.0463
Epoch 9/100
3/3 [==============================] - 0s 15ms/step - loss: 0.0451
Epoch 10/100
3/3 [==============================] - 0s 11ms/step - loss: 0.0404
Epoch 11/100
3/3 [==============================] - 0s 14ms/step - loss: 0.0366
Epoch 12/100
3/3 [==============================] - 0s 16ms/step - loss: 0.0344
Epoch 13/100
3/3 [==============================]

In [10]:
loss = model.evaluate(X_test, y_test)
print('Test Loss:', loss)

1/1 [==============================] - 1s 1s/step - loss: 0.0165
Test Loss: 0.016530103981494904


In [11]:
X_future = X_test[-10:]

# Predviđanje kretanja temperature za budući period
future_predictions_normalized = model.predict(X_future)
future_predictions = scaler.inverse_transform(future_predictions_normalized)
print('Future Predictions:', future_predictions)

1/1 [==============================] - 1s 655ms/step
Future Predictions: [[11.534826]
 [11.682248]
 [11.769399]
 [11.254004]
 [12.377547]
 [12.98561 ]
 [11.89572 ]
 [11.826719]
 [12.212879]
 [12.372303]]


In [12]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Izračunaj predviđanja za testni skup podataka
predictions = model.predict(X_test)

# Izračunaj MAE
mae = mean_absolute_error(y_test, predictions)

# Izračunaj MSE
mse = mean_squared_error(y_test, predictions)

print('Mean Absolute Error (MAE):', mae)
print('Mean Squared Error (MSE):', mse)

1/1 [==============================] - 0s 31ms/step
Mean Absolute Error (MAE): 0.09685239595792527
Mean Squared Error (MSE): 0.016530108866910666


In [13]:
time_steps = 4

In [14]:
time_series_summer = df['leto'].values.astype(float)
scaler = MinMaxScaler(feature_range=(0, 1))
time_series_normalized = scaler.fit_transform(time_series_summer.reshape(-1, 1))

In [15]:
X, y = create_dataset(time_series_normalized, time_steps)

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [17]:
model_summer = Sequential([
    GRU(50, input_shape=(time_steps, 1)),
    Dense(1)
])

model_summer.compile(optimizer='adam', loss='mean_squared_error')

model_summer.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=1)

Epoch 1/100
4/4 [==============================] - 3s 10ms/step - loss: 0.1974
Epoch 2/100
4/4 [==============================] - 0s 8ms/step - loss: 0.1222
Epoch 3/100
4/4 [==============================] - 0s 9ms/step - loss: 0.0712
Epoch 4/100
4/4 [==============================] - 0s 12ms/step - loss: 0.0414
Epoch 5/100
4/4 [==============================] - 0s 8ms/step - loss: 0.0295
Epoch 6/100
4/4 [==============================] - 0s 7ms/step - loss: 0.0323
Epoch 7/100
4/4 [==============================] - 0s 8ms/step - loss: 0.0354
Epoch 8/100
4/4 [==============================] - 0s 8ms/step - loss: 0.0346
Epoch 9/100
4/4 [==============================] - 0s 7ms/step - loss: 0.0320
Epoch 10/100
4/4 [==============================] - 0s 7ms/step - loss: 0.0297
Epoch 11/100
4/4 [==============================] - 0s 7ms/step - loss: 0.0289
Epoch 12/100
4/4 [==============================] - 0s 10ms/step - loss: 0.0295
Epoch 13/100
4/4 [==============================] - 0s 8ms

In [18]:
loss = model_summer.evaluate(X_test, y_test)
print('Test Loss - summer:', loss)

1/1 [==============================] - 1s 702ms/step - loss: 0.0212
Test Loss - summer: 0.02116621658205986


In [19]:
X_future = X_test[-10:]


future_predictions_normalized = model_summer.predict(X_future)
future_predictions = scaler.inverse_transform(future_predictions_normalized)
print('Future Predictions:- summer', future_predictions)

1/1 [==============================] - 1s 589ms/step
Future Predictions:- summer [[21.162203]
 [22.618095]
 [20.744593]
 [21.959127]
 [20.78077 ]
 [21.8364  ]
 [22.383196]
 [21.291876]
 [21.268307]
 [21.127409]]


In [20]:

predictions_summer = model_summer.predict(X_test)


mae = mean_absolute_error(y_test, predictions_summer)


mse = mean_squared_error(y_test, predictions_summer)

print('Mean Absolute Error (MAE)- summer:', mae)
print('Mean Squared Error (MSE)- summer:', mse)

1/1 [==============================] - 0s 29ms/step
Mean Absolute Error (MAE)- summer: 0.10591679726312855
Mean Squared Error (MSE)- summer: 0.02116621621378769


In [21]:
time_series_winter = df['zima'].values.astype(float)
scaler = MinMaxScaler(feature_range=(0, 1))
time_series_normalized = scaler.fit_transform(time_series_winter.reshape(-1, 1))

X, y = create_dataset(time_series_normalized, time_steps)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [22]:
model_winter = Sequential([
    GRU(50, input_shape=(time_steps, 1)),
    Dense(1)
])

model_winter.compile(optimizer='adam', loss='mean_squared_error')

model_winter.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=1)

Epoch 1/100
4/4 [==============================] - 4s 10ms/step - loss: 0.3492
Epoch 2/100
4/4 [==============================] - 0s 10ms/step - loss: 0.2406
Epoch 3/100
4/4 [==============================] - 0s 8ms/step - loss: 0.1608
Epoch 4/100
4/4 [==============================] - 0s 10ms/step - loss: 0.0978
Epoch 5/100
4/4 [==============================] - 0s 12ms/step - loss: 0.0614
Epoch 6/100
4/4 [==============================] - 0s 8ms/step - loss: 0.0386
Epoch 7/100
4/4 [==============================] - 0s 9ms/step - loss: 0.0358
Epoch 8/100
4/4 [==============================] - 0s 11ms/step - loss: 0.0410
Epoch 9/100
4/4 [==============================] - 0s 9ms/step - loss: 0.0437
Epoch 10/100
4/4 [==============================] - 0s 8ms/step - loss: 0.0423
Epoch 11/100
4/4 [==============================] - 0s 10ms/step - loss: 0.0393
Epoch 12/100
4/4 [==============================] - 0s 12ms/step - loss: 0.0367
Epoch 13/100
4/4 [==============================] - 0s

In [23]:
loss_winter = model_winter.evaluate(X_test, y_test)
print('Test Loss:- winter', loss)

1/1 [==============================] - 1s 779ms/step - loss: 0.0417
Test Loss:- winter 0.02116621658205986


In [24]:
X_future = X_test[-10:]


future_predictions_normalized = model_winter.predict(X_future)
future_predictions = scaler.inverse_transform(future_predictions_normalized)
print('Future Predictions: - winter', future_predictions)

1/1 [==============================] - 1s 816ms/step
Future Predictions: - winter [[1.002137  ]
 [2.1959736 ]
 [1.8952364 ]
 [1.0742562 ]
 [1.550325  ]
 [1.654435  ]
 [1.7435409 ]
 [1.6172186 ]
 [0.68163204]
 [1.76853   ]]


In [25]:

predictions_winter = model_winter.predict(X_test)


mae = mean_absolute_error(y_test, predictions_winter)


mse = mean_squared_error(y_test, predictions_winter)

print('Mean Absolute Error (MAE)- winter:', mae)
print('Mean Squared Error (MSE):- winter', mse)

1/1 [==============================] - 0s 44ms/step
Mean Absolute Error (MAE)- winter: 0.16023356548003728
Mean Squared Error (MSE):- winter 0.0417278757141628
